#### Implementing regularization on Fashion MNIST dataset using MLP

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras # Don't import keras from tensorflow like (from tensorflow import keras)
import seaborn as sns

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Reading train and test data
train_data = pd.read_csv('fashion-mnist_train.csv')
test_data = pd.read_csv('fashion-mnist_test.csv')

In [3]:
# Class names
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Making validation data out of test data
val_data = test_data.iloc[:5000,:]
test_data = test_data.iloc[5000:,:]

# Fetching the labels
train_labels = train_data.label
val_labels = val_data.label
test_labels = test_data.label

# Reshaping training data
train_images = train_data.iloc[:,1:].values.reshape(60000, 28, 28)

# Reshaping validation data
val_images = val_data.iloc[:,1:].values.reshape(5000, 28, 28)

# Reshaping test data
test_images = test_data.iloc[:,1:].values.reshape(5000, 28, 28)

# Scaling data in the range of 0-1
train_images = train_images/255.0
val_images = val_images/255.0
test_images = test_images/255.0

In [4]:
def regularization(list_of_regs, reg_names):
    for reg_init in range(len(list_of_regs)):        
        # Writing structure of model
        model = keras.Sequential([
            keras.layers.Flatten(input_shape=(28, 28)), 
            keras.layers.Dense(256, kernel_regularizer=list_of_regs[reg_init], activation=tf.nn.relu), 
            keras.layers.Dense(10, activation=tf.nn.softmax) 
        ])      

        # Defining parameters like optmizer, loss function and evaluating metric
        model.compile(loss='sparse_categorical_crossentropy', # 
                      optimizer=keras.optimizers.Adam(), # Learning rate and momentum can be passed inside optimizer
                      metrics=['accuracy'])

        # Traning the model and writing log file for TensorBoard
        
        logdir = r'regularization\\' + reg_names[reg_init] # Each log file needs to be written in a distinct directory. (Mandatory)        

        # To store tensorboard graph logs
        cb_tensorboard = keras.callbacks.TensorBoard(log_dir=logdir, write_graph=False) 

        epochs = 5
        print('Model building using ' + reg_names[reg_init] + ' Regularization')
        model.fit(train_images, train_labels, epochs=epochs, 
                    validation_data=(val_images, val_labels),
                    callbacks=[cb_tensorboard])
        print('Model built successfully.')
        print('')

In [5]:
# Value of lambda can be calculated using cross-validation
list_of_regs = [keras.regularizers.l2(), keras.regularizers.l1(), keras.regularizers.l1_l2()] 
reg_names = ['Ridge.L2', 'Lasso.L1', 'Elastic.Net.L1.L2']
regularization(list_of_regs, reg_names)

Model building using Ridge.L2 Regularization
Train on 60000 samples, validate on 5000 samples
Epoch 1/5
60000/60000 [==============================] - 14s 229us/step - loss: 0.9248 - acc: 0.7883 - val_loss: 0.6838 - val_acc: 0.8050
Epoch 2/5
60000/60000 [==============================] - 13s 213us/step - loss: 0.6766 - acc: 0.8036 - val_loss: 0.6855 - val_acc: 0.8118
Epoch 3/5
60000/60000 [==============================] - 13s 216us/step - loss: 0.6445 - acc: 0.8134 - val_loss: 0.5912 - val_acc: 0.8368
Epoch 4/5
60000/60000 [==============================] - 11s 191us/step - loss: 0.6159 - acc: 0.8196 - val_loss: 0.5884 - val_acc: 0.8380
Epoch 5/5
60000/60000 [==============================] - 12s 206us/step - loss: 0.6005 - acc: 0.8238 - val_loss: 0.5984 - val_acc: 0.8282
Model built successfully.

Model building using Lasso.L1 Regularization
Train on 60000 samples, validate on 5000 samples
Epoch 1/5
60000/60000 [==============================] - 13s 213us/step - loss: 2.8723 - acc: 0

#### Open tensorboard in `regularization` directory